This notebook assumes that you have a running instance of Neo4J Aura. You can create a free one by heading over to https://neo4j.com/

Once you have your instance, save its id as `neo4j_instance_id` and its password as `neo4j_password` as Colab secrets and grant this notebook access to said secrets.

In [1]:
!pip install neo4j yfiles_jupyter_graphs_for_neo4j pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.3/312.3 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 40.4 MB/s eta 0:00:00


In [2]:
from neo4j import GraphDatabase

In [3]:
from google.colab import userdata
neo4j_instance_id = userdata.get('neo4j_instance_id')
neo4j_password = userdata.get('neo4j_password')

driver = GraphDatabase.driver(f"neo4j+s://{neo4j_instance_id}.databases.neo4j.io:7687", auth=("neo4j", neo4j_password))

In [4]:
def insert_data(tx):
    query = """
      CREATE (A:Location {name: 'A'})
      CREATE (B:Location {name: 'B'})
      CREATE (C:Location {name: 'C'})
      CREATE (D:Location {name: 'D'})

      CREATE (A)-[:CONNECTED {weight: 1}]->(B)
      CREATE (A)-[:CONNECTED {weight: 2}]->(C)
      CREATE (A)-[:CONNECTED {weight: 4}]->(D)
      CREATE (B)-[:CONNECTED {weight: 2}]->(C)
      CREATE (B)-[:CONNECTED {weight: 3}]->(D)
      CREATE (C)-[:CONNECTED {weight: 1}]->(D)

    RETURN *
    """
    tx.run(query)

with driver.session() as session:
    session.execute_write(insert_data)

In [5]:
from yfiles_jupyter_graphs_for_neo4j import Neo4jGraphWidget

g = Neo4jGraphWidget(driver)

def show_graph(driver):
    query = """
    MATCH (n)-[r]->(m)
    RETURN n, r, m
    """
    g.show_cypher(query)

show_graph(driver)

GraphWidget(layout=Layout(height='500px', width='100%'))

In [6]:
from google.colab import output
output.enable_custom_widget_manager()

Support for third party widgets will remain active for the duration of the session. To disable support:

In [ ]:
from google.colab import output
output.disable_custom_widget_manager()

In [7]:
import pandas as pd

def run_dijkstra(driver):
    query = """
      WITH 'A' AS startName, 'D' AS endName
      MATCH (start:Location {name: startName}), (end:Location {name: endName})
      CALL apoc.algo.dijkstra(start, end, 'CONNECTED', 'weight') YIELD path, weight
      RETURN
        nodes(path) AS nodes,
        relationships(path) AS edges,
        weight
    """
    with driver.session() as session:
        result = session.run(query)
        return [record.data() for record in result]

results = run_dijkstra(driver)
pd.DataFrame(results)

,nodes,edges,weight
0,"[{'name': 'A'}, {'name': 'C'}, {'name': 'D'}]","[({'name': 'A'}, CONNECTED, {'name': 'C'}), ({...",3.0
